<a href="https://colab.research.google.com/github/Henil21/Intro_to_computer_vision_and_CNN/blob/main/Transfer_learning_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

There are two main benefits to using transfer learning:

1. Can leverage an existing neural network architecture proven to work on problems similar to our own.


2. Can leverage a working neural network architecture which has already learned patterns on similar data to our own. This often results in achieving great results with less custom data.

In [3]:
# lets check we are using gpu or not
!nvidia-smi

Thu Nov 17 12:50:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# get data (10$ if 10 food class)
import zipfile
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
zip_ref=zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

--2022-11-17 12:50:01--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 74.125.68.128, 142.250.4.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  21.4MB/s    in 8.6s    

2022-11-17 12:50:10 (18.8 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [5]:
import os
for dirpath,dirname,filenames in os.walk("10_food_classes_10_percent"):
  print(f"there are {len(dirname)} directories and {len(filenames)} images in '{dirpath}'")

there are 2 directories and 0 images in '10_food_classes_10_percent'
there are 10 directories and 0 images in '10_food_classes_10_percent/test'
there are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'
there are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'
there are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'
there are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'
there are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'
there are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'
there are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'
there are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'
there are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'
there are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza

## Creating loader()
we will use `ImageDataGenerator` class to load in our images in batches

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE=(224,224)
BATCH_SIZE=32
train_dir="10_food_classes_10_percent/train/"
test_dir="10_food_classes_10_percent/test/"

train_datagen=ImageDataGenerator(rescale=1/255)
test_datagen=ImageDataGenerator(rescale=1/255)


train_datagen=train_datagen.flow_from_directory(train_dir,
                                 target_size=IMAGE_SHAPE,
                                 batch_size=BATCH_SIZE,
                                 class_mode="categorical")

print("testing images:")
test_data=test_datagen.flow_from_directory(test_dir,
                                           target_size=IMAGE_SHAPE,
                                           batch_size=BATCH_SIZE,
                                           class_mode='categorical')

Found 750 images belonging to 10 classes.
testing images:
Found 2500 images belonging to 10 classes.


## Setting Up callback 🔙(things to run while our model trains)

callback are extra functionality we can add to our models to performed during or after training .some of the most popular callbacks:

* Experiment tracking with TensorBoard - log the performance of multiple models and then view and compare these models in a visual way on TensorBoard (a dashboard for inspecting neural network parameters). Helpful to compare the results of different models on your data.


* Model checkpointing - save your model as it trains so you can stop training if needed and come back to continue off where you left. Helpful if training takes a long time and can't be done in one sitting.


* Early stopping - leave your model training for an arbitrary amount of time and have it stop training automatically when it ceases to improve. Helpful when you've got a large dataset and don't know how long training will take.

In [7]:
# create TensorBoard callback (functionized because we need to create a new model)
import datetime
import tensorflow as tf
def create_tensorboard_callback(dir_name,_experiment_name):
  log_dir=dir_name+'/'+experiment_name+"/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"saving tfboard log file to{log_dir}")
  return  tensorboard_callback